In [1]:
import json
import cv2
import os
import matplotlib.pyplot as plt
import shutil

In [2]:
input_path = "D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/project-1-at-2024-05-18-14-40-69c0cd42/"
output_path = "D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/dataset_yolo/"

In [3]:
f = open(input_path+'result.json')
data = json.load(f)
f.close()

In [4]:
data.keys()

dict_keys(['images', 'categories', 'annotations', 'info'])

In [5]:
data['images'][2]

{'width': 192,
 'height': 320,
 'id': 2,
 'file_name': 'gs://vp-ml-final-dataset/dataset/Capsule_defect/002.png'}

In [6]:
data['categories']

[{'id': 0, 'name': 'Capsule_Defect'},
 {'id': 1, 'name': 'Capsule_Normal'},
 {'id': 2, 'name': 'Softgel_Defect'},
 {'id': 3, 'name': 'Softgel_Normal'},
 {'id': 4, 'name': 'Tablet_Defect'},
 {'id': 5, 'name': 'Tablet_Normal'}]

In [7]:
data['annotations'][2]

{'id': 2,
 'image_id': 1,
 'category_id': 0,
 'segmentation': [],
 'bbox': [35.307781649245065,
  13.008130081300811,
  130.4529616724739,
  273.1707317073171],
 'ignore': 0,
 'iscrowd': 0,
 'area': 35635.930993456284}

In [6]:
file_names = []

def load_images_from_subfolders(gs_directory,main_folder):
    count = 0
    for subfolder in os.listdir(main_folder):
        folder_path = os.path.join(main_folder, subfolder)
        if os.path.isdir(folder_path):  # Check if it's a directory
            for filename in os.listdir(folder_path):
                source = os.path.join(folder_path, filename)
                os.makedirs(os.path.join(output_path,"images_raw/"+subfolder), exist_ok=True)
                destination = os.path.join(output_path, "images_raw/"+subfolder+f"/img{count}.png")
                
                try:
                    shutil.copy(source, destination)
                    print(f"File copied successfully: {filename}")
                except shutil.SameFileError:
                    print("Source and destination represents the same file.")
                except Exception as e:
                    print(f"Error occurred while copying file: {e}")

                file_names.append(gs_directory+subfolder+"/"+filename)
                count += 1

# Example usage:
main_folder = 'dataset/'
gs_directory_main_folder = 'gs://vp-ml-final-dataset/dataset/'
load_images_from_subfolders(gs_directory_main_folder,main_folder)

File copied successfully: 000.png
File copied successfully: 001.png
File copied successfully: 002.png
File copied successfully: 005.png
File copied successfully: 007.png
File copied successfully: 008.png
File copied successfully: 011.png
File copied successfully: 017.png
File copied successfully: 020.png
File copied successfully: 026.png
File copied successfully: 032.png
File copied successfully: 033.png
File copied successfully: 039.png
File copied successfully: 042.png
File copied successfully: 044.png
File copied successfully: 048.png
File copied successfully: 049.png
File copied successfully: 050.png
File copied successfully: 051.png
File copied successfully: 057.png
File copied successfully: 061.png
File copied successfully: 065.png
File copied successfully: 068.png
File copied successfully: 072.png
File copied successfully: 077.png
File copied successfully: 078.png
File copied successfully: 079.png
File copied successfully: 081.png
File copied successfully: 083.png
File copied su

In [7]:
def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None

In [8]:
def get_img(filename):
  for img in data['images']:
    if img['file_name'] == filename:
      return img

In [17]:
count = 0

for filename in file_names:
  # Extracting image 
  img = get_img(filename)
  img_id = img['id']
  img_w = img['width']
  img_h = img['height']

  # Get Annotations for this image
  img_ann = get_img_ann(img_id)

  if img_ann:
    # Opening file for current image
    file_object = open(f"{output_path}labels_raw/img{count}.txt", "a")

    for ann in img_ann:
#     current_category = ann['category_id'] - 1 # As yolo format labels start from 0 
      current_category = ann['category_id'] # the json label already 0,1,2,etc
      current_bbox = ann['bbox']
      x = current_bbox[0]
      y = current_bbox[1]
      w = current_bbox[2]
      h = current_bbox[3]
      
      # Finding midpoints
      x_centre = (x + (x+w))/2
      y_centre = (y + (y+h))/2
      
      # Normalization
      x_centre = x_centre / img_w
      y_centre = y_centre / img_h
      w = w / img_w
      h = h / img_h
      
      # Limiting upto fix number of decimal places
      x_centre = format(x_centre, '.6f')
      y_centre = format(y_centre, '.6f')
      w = format(w, '.6f')
      h = format(h, '.6f')
          
      # Writing current object 
      file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")

    file_object.close()
    count += 1  # This should be outside the if img_ann block.

In [12]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the path to the main dataset directory
data_dir = './dataset_yolo/images_raw'
label_dir = './dataset_yolo/labels_raw'
data_dest_dir = './dataset_yolo/images'
label_dest_dir = './dataset_yolo/labels'
train_dir ='/train'
val_dir = '/val'
test_dir = '/test'


# Ensure the train, validation, and test directories exist


def split_data_2(data_dir, label_dir,data_dest_dir, label_dest_dir, train_dir, val_dir, test_dir, train_size=0.7, val_size = 0.2, test_size=0.1):

    
    os.makedirs(data_dest_dir+train_dir, exist_ok=True)
    os.makedirs(data_dest_dir+val_dir, exist_ok=True)
    os.makedirs(data_dest_dir+test_dir, exist_ok=True)
    os.makedirs(label_dest_dir+train_dir, exist_ok=True)
    os.makedirs(label_dest_dir+val_dir, exist_ok=True)
    os.makedirs(label_dest_dir+test_dir, exist_ok=True)
    # Iterate over each class directory in the source directory
    for class_dir in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):

            # Get all file names in the class directory
            files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
            #print(files)

            # Split the data
            train_files, temp_files = train_test_split(files, train_size=train_size, random_state=42)
            val_files, test_files = train_test_split(temp_files, test_size=test_size / (1 - train_size), random_state=42)

            train_labels = [f.replace('.png', '.txt') for f in train_files]
            val_labels = [f.replace('.png', '.txt') for f in val_files]
            test_labels = [f.replace('.png', '.txt') for f in test_files]
            #print(train_labels)
            # Function to copy files
            def copy_files(files, dest_dir):
                for file in files:
                    shutil.copy(os.path.join(class_path, file), os.path.join(dest_dir, file))
                    
            def copy_labels(files, dest_dir):
                for file in files:
                    shutil.copy(os.path.join(label_dir, file), os.path.join(dest_dir, file))

            # Copy files to their respective directories
            copy_files(train_files, data_dest_dir+train_dir)
            copy_files(val_files, data_dest_dir+val_dir)
            copy_files(test_files, data_dest_dir+test_dir)
            copy_labels(train_labels, label_dest_dir+train_dir)
            copy_labels(val_labels, label_dest_dir+val_dir)
            copy_labels(test_labels, label_dest_dir+test_dir)
            
            
# Call the function to split the data
split_data_2(data_dir,label_dir,data_dest_dir, label_dest_dir, train_dir, val_dir, test_dir)

['img0.png', 'img1.png', 'img10.png', 'img100.png', 'img101.png', 'img102.png', 'img103.png', 'img104.png', 'img105.png', 'img106.png', 'img107.png', 'img108.png', 'img109.png', 'img11.png', 'img110.png', 'img111.png', 'img112.png', 'img113.png', 'img114.png', 'img115.png', 'img116.png', 'img117.png', 'img118.png', 'img119.png', 'img12.png', 'img120.png', 'img121.png', 'img122.png', 'img123.png', 'img124.png', 'img125.png', 'img126.png', 'img127.png', 'img128.png', 'img129.png', 'img13.png', 'img130.png', 'img131.png', 'img132.png', 'img133.png', 'img134.png', 'img135.png', 'img136.png', 'img137.png', 'img138.png', 'img139.png', 'img14.png', 'img140.png', 'img141.png', 'img142.png', 'img143.png', 'img144.png', 'img145.png', 'img146.png', 'img147.png', 'img148.png', 'img149.png', 'img15.png', 'img150.png', 'img151.png', 'img152.png', 'img153.png', 'img154.png', 'img155.png', 'img156.png', 'img157.png', 'img158.png', 'img159.png', 'img16.png', 'img160.png', 'img161.png', 'img17.png', 'im